```
On a staircase, the i-th step has some non-negative cost cost[i] assigned (0 indexed).

Once you pay the cost, you can either climb one or two steps. You need to find minimum cost to reach the top of the floor, and you can either start from the step with index 0, or the step with index 1.

Example 1:
Input: cost = [10, 15, 20]
Output: 15
Explanation: Cheapest is start on cost[1], pay that cost and go to the top.
Example 2:
Input: cost = [1, 100, 1, 1, 1, 100, 1, 1, 100, 1]
Output: 6
Explanation: Cheapest is start on cost[0], and only step on 1s, skipping cost[3].
Note:
cost will have a length in the range [2, 1000].
Every cost[i] will be an integer in the range [0, 999].
```

In [2]:
class Solution:
    def minCostClimbingStairs(self, cost):
        """
        :type cost: List[int]
        :rtype: int
        """
        lens = len(cost)
        cost.append(0)
        cost.append(0)
        ret1 = ret2 = 0
        
        def ClimbingStairs(_cost,start,end):
            if start == end - 1:
                return min(_cost[start],_cost[start+1]) if start ==0 else _cost[start]
            if start == end: return _cost[start]
            else:
                a = (
                    ClimbingStairs(cost,start+1,end),
                    _cost[start]+ ClimbingStairs(cost,start+2,end),
                    _cost[start+1]+ ClimbingStairs(cost,start+2,end))
                return min(a) if start == 0 else  \
                    _cost[start] + \
                    min(ClimbingStairs(cost,start+1,end),ClimbingStairs(cost,start+2,end))
            
        return ClimbingStairs(cost,0,lens-1)
                

严重的超时了, 要想想其他策略,动态规划的算法:

In [5]:
class Solution:
    def minCostClimbingStairs(self, cost):
        """
        :type cost: List[int]
        :rtype: int
        """    
        prevPrev, prev  = cost[0], cost[1]
        for i in range(2, len(cost)):
            cur = min(prevPrev, prev) + cost[i]
            prevPrev, prev = prev, cur
        return min(prevPrev, prev)


子问题是: 到第n步的最小代价为dp[n]
```   
第1步: cost[0]
第2步: cost[1] 
第n步: cost[n] + min(dp[n-1],dp[n-2])
```
    
因为有2种可能的start, 因此需要同时更新2个  

### 插播DP的理解:
例子:
```
最长上升子序列LIS:

输入n及一个长度为n的数列，求出此序列的最长上升子序列长度。
上升子序列指的是对于任意的i<j都满足ai<aj的子序列。（1<=n<=1000,0<=ai<=1000000）
```
三个原则实施DP:
```
1. 可划分阶段
2. 最优化原理,即子问题的局部最优将导致整个问题的全局最优，即问题具有最优子结构的性质,
   也就是说一个问题的最优解只取决于其子问题的最优解
3. 无后效性,即当前的状态只和过去有关,和未来无关,有点像线性时不变里的因果性
```

划分法1:
    
```
最长上升子序列可能以任何位置作为结尾! 这就是`划分子问题`(方式并不唯一!), 即: dp[n]是以a[n]结尾的最长子序列的长度;

dp[n]可以递推, 其值可能和dp[0],dp[1],...dp[n-1]有关, 这是`无后效性`

所有的dp[n]求得时,全局问题自然得解,即max(dp)
```

dp的值要么为1,要么为其他,因此初始值全部为1即可


```
step: 0 , dp[0] = 1
step: i, 对于j in range(1,i) a[j]是dp[j]代表的最长子序列的结尾, if a[i] > a[j],此时可以形成新的最长子序列 长度为a[j]+1,
    找到所有dp[j]中,可能的最大的dp[j]+1, 用来更新dp[i]
```

In [10]:
class Solution:
    def maxSubArrayLength(self,nums):
        dp = []
        [dp.append(1) for i in range(0,len(nums))]
        print(dp)
        for i in range(1,len(nums)):
            for j in range(1,i):
                dp[i] = max(dp[i],dp[j]+1) if nums[i] > nums[j] else dp[i]
        print(dp)
        return max(dp)


In [11]:
Solution().maxSubArrayLength([4,2,3,1,100,2,3,4,5,6,7,6,6,8])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 2, 1, 3, 2, 3, 4, 5, 6, 7, 6, 6, 8]


8

划分法2:
    再来一种划分子问题的思路: 
```
dp[i]: 长度为i+1的上升子序列中末尾元素的最小值（不存在的话就是INF）。 i是递归的变量 最大长度i+1 == len(nums)

不存在,则标记为初值INF

100 99 91 92 93 94 95 1 2 3 4 5 3 
100 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
99 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 92 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 92 93 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 92 93 94 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 92 93 94 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 92 93 94 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 93 94 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 3 94 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 3 4 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 3 4 5 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 3 4 5 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
5

```

In [13]:
class Solution:
    def maxSubArrayLength(self,nums):
        import sys
        dp = []
        INTMAX =999
        [dp.append(INTMAX) for i in range(0,len(nums))]
#         print(dp)
        
        dp[0] = nums[0]
        for j in range(1,len(nums)):
            for i in range(0,len(nums)):
                if nums[j] < dp[i]: 
                    dp[i] = min(nums[j],dp[i])
                    break
            print(dp)
            
        return dp.index(999)


In [14]:
Solution().maxSubArrayLength([100, 99 ,91, 92 ,93,94 , 1, 2 ,3 ,4, 5, 3 ])

[99, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[91, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[91, 92, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[91, 92, 93, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[91, 92, 93, 94, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 92, 93, 94, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 93, 94, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 3, 94, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 3, 4, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 3, 4, 5, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 3, 3, 5, 999, 999, 999, 999, 999, 999, 999]


5

扩充:
    
求两个字符串的公共子序列的最大长度. 公共子串是相同的子串.
```
dp[i-1][j-1]为a为长度i,b为长度j时的公共子串的长度
dp[i]d[j] = dp[i-1][j-1] m+ 1 if a[i-1]==b[j-1]
否则 dp[i]d[j]  = max(dp[i]d[j-1] ,dp[i]d[j-1] )
```

In [16]:
class Solution:
    def maxSubArrayLength(self,a,b):
        lena = len(a)
        lenb = len(b)
        dp = []
        for i in range(0,lena):
            dp.append([])
            for j in range(0,lenb):
                dp[i].append(0)
        print(dp)
        dp[0][0] = 1 if a[0] == b[0] else 0
        for i in range(0,lena-1):
            for j in range(0,lenb-1):
                dp[i+1][j+1] = dp[i][j] + 1 if a[i] == b[j] else max(dp[i+1][j],dp[i][j+1])
        print(dp)
        return  dp[lena-1][lenb-1]       

In [17]:
Solution().maxSubArrayLength([2,3],[2,3,4])

[[0, 0, 0], [0, 0, 0]]
[[1, 0, 0], [0, 2, 2]]


2

## 674. Longest Continuous Increasing Subsequence